In [1]:
import nltk
nltk.download("popular")
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import string
from nltk.corpus import stopwords
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [2]:
data = pd.read_csv("mitdataset.txt",delimiter="\t")
data.head()


,A person on a horse jumps over a broken down airplane.,"A person is at a diner, ordering an omelette.",0
0,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1.0
1,Children smiling and waving at camera,There are children present,1.0
2,Children smiling and waving at camera,The kids are frowning,0.0
3,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0.0
4,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,1.0


In [3]:
data.columns = [ "source_text", "plagiarized_text", "label"]
data.head()

,source_text,plagiarized_text,label
0,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1.0
1,Children smiling and waving at camera,There are children present,1.0
2,Children smiling and waving at camera,The kids are frowning,0.0
3,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0.0
4,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,1.0


In [4]:
data["label"].value_counts()

,count
label,
0.0,9913
1.0,9899


In [5]:
data.shape

(19813, 3)

In [6]:
data["source_text"] = data["source_text"].astype(str)
data["plagiarized_text"] = data["plagiarized_text"].astype(str)

In [7]:
def preprocess_text(text):
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.lower()
    stop_words = set(stopwords.words("english"))
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text
data["source_text"] = data["source_text"].apply(preprocess_text)
data["plagiarized_text"] = data["plagiarized_text"].apply(preprocess_text)


In [8]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data["source_text"] + " " + data["plagiarized_text"])

In [9]:
y = data["label"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
import numpy as np

print("NaN values in y_train:", np.isnan(y_train).sum())
print("NaN values in y_test:", np.isnan(y_test).sum())


NaN values in y_train: 1
NaN values in y_test: 0


In [13]:
X_train = X_train[~np.isnan(y_train)]
y_train = y_train[~np.isnan(y_train)]

X_test = X_test[~np.isnan(y_test)]
y_test = y_test[~np.isnan(y_test)]


In [14]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix")
print(cm)

Accuracy: 0.5674993691647742
Classification Report:
              precision    recall  f1-score   support

         0.0       0.57      0.55      0.56      1983
         1.0       0.57      0.58      0.57      1980

    accuracy                           0.57      3963
   macro avg       0.57      0.57      0.57      3963
weighted avg       0.57      0.57      0.57      3963

Confusion Matrix
[[1097  886]
 [ 828 1152]]


In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

classification_rep = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.5632096896290689
Classification Report:
              precision    recall  f1-score   support

         0.0       0.57      0.49      0.53      1983
         1.0       0.55      0.64      0.59      1980

    accuracy                           0.56      3963
   macro avg       0.56      0.56      0.56      3963
weighted avg       0.56      0.56      0.56      3963

Confusion Matrix:
[[ 971 1012]
 [ 719 1261]]


In [16]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.4925561443350997
Classification Report:
              precision    recall  f1-score   support

         0.0       0.49      0.44      0.46      1983
         1.0       0.49      0.55      0.52      1980

    accuracy                           0.49      3963
   macro avg       0.49      0.49      0.49      3963
weighted avg       0.49      0.49      0.49      3963

Confusion Matrix:
[[ 868 1115]
 [ 896 1084]]


In [17]:
from sklearn.svm import SVC


model = SVC(kernel='linear', random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.5546303305576583
Classification Report:
              precision    recall  f1-score   support

         0.0       0.56      0.51      0.53      1983
         1.0       0.55      0.60      0.57      1980

    accuracy                           0.55      3963
   macro avg       0.56      0.55      0.55      3963
weighted avg       0.56      0.55      0.55      3963

Confusion Matrix:
[[1012  971]
 [ 794 1186]]


In [18]:
import pickle

pickle.dump(model,open("model.pkl",'wb'))
pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.pkl','wb'))

In [19]:
model = pickle.load(open('model.pkl','rb'))
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl','rb'))

In [20]:
def detect(input_text):
    vectorized_text = tfidf_vectorizer.transform([input_text])
    result = model.predict(vectorized_text)
    return "Plagiarim Detected" if result[0] == 1 else "No Plagiarism"

In [21]:
input_text = 'Researchers have discovered a new species of butterfly in the Amazon rainforest.'
detect(input_text)

'Plagiarim Detected'

In [26]:
input_text = 'There are children present'
detect(input_text)

'Plagiarim Detected'